id 부여

In [ ]:
import os
import pandas as pd
import mysql.connector

#DB 연결
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1234',
    database='premo',
)
cursor = conn.cursor()

#team_name 추출
team_set = set()

data_root = './datas/2'

for folder in os.listdir(data_root):
    folder_path = os.path.join(data_root, folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith('.csv') and 'teams' in file.lower():
                file_path = os.path.join(folder_path, file)
                try:
                    df = pd.read_csv(file_path)
                    
                    if 'common_name' in df.columns:
                        team_set.update(df['common_name'].dropna().unique())
                except Exception as e:
                    print(f"X Error in {file_path}: {e}")
                    
""" #db에 삽입(중복 방지)
for team in team_set:
    cursor.execute("insert into teams (team_name) values (%s)", (team,)) """
    
#완료
conn.commit()
cursor.close()
conn.close()

1. matches_processsed.csv

In [39]:
import pandas as pd
import os

data_root = './datas/2'
data_root2 = './datas/1'

2.

In [40]:
import pandas as pd
import os

# 설정
matches_drop = ['timestamp', 'status', 'attendance', 'referee', 'stadium_name', 'Game Week']
interval_labels = ['0_9', '10_19', '20_29', '30_39', '40_49', '50_59', '60_69', '70_79', '80_89', '90_99', '100_plus']

# 골 타이밍 전처리
def process_goal_timings(goal_str):
    if pd.isna(goal_str) or not isinstance(goal_str, str):
        return []
    goal_str = goal_str.replace("'", "")
    return [int(x) for x in goal_str.split(',') if x.strip().isdigit()]

# One-hot 인코딩
def one_hot_encode_goals(goal_list):
    bins = [0] * 11
    for g in goal_list:
        idx = min(g // 10, 10)
        bins[idx] = 1
    return bins

# 승패 판단
def determine_result(home_goals, away_goals):
    if home_goals > away_goals:
        return 'win', 'lose'
    elif home_goals < away_goals:
        return 'lose', 'win'
    else:
        return 'draw', 'draw'

# 폴더 순회
for root, dirs, files in os.walk(data_root):
    for file in files:
        if file.endswith('.csv') and 'matches' in file.lower():
            file_path = os.path.join(root, file)
            try:
                print(f'[처리중] {file_path}')
                df = pd.read_csv(file_path)

                # 불필요한 컬럼 삭제
                df_processed = df.drop(columns=[col for col in matches_drop if col in df.columns])

                # one-hot 인코딩
                for team in ['home', 'away']:
                    col = f"{team}_team_goal_timings"
                    if col in df_processed.columns:
                        one_hot_series = df_processed[col].apply(lambda x: one_hot_encode_goals(process_goal_timings(x)))
                        one_hot_df = pd.DataFrame(one_hot_series.tolist(), columns=[f"{team}_goal_{label}" for label in interval_labels])
                        df_processed = df_processed.drop(columns=[col])
                        df_processed = pd.concat([df_processed, one_hot_df], axis=1)

                # 승패 컬럼 추가
                if 'home_team_goal_count' in df_processed.columns and 'away_team_goal_count' in df_processed.columns:
                    df_processed['home_result'], df_processed['away_result'] = zip(*df_processed.apply(
                        lambda row: determine_result(row['home_team_goal_count'], row['away_team_goal_count']), axis=1
                    ))

                # 저장
                new_path = os.path.join(root, file.replace('.csv', '_processed.csv'))
                df_processed.to_csv(new_path, index=False)
                print(f"[완료] matches: {new_path}")
            except Exception as e:
                print(f"[오류] {file_path}: {e}")


[처리중] ./datas/2\1617\england-premier-league-matches-2016-to-2017-stats.csv
[완료] matches: ./datas/2\1617\england-premier-league-matches-2016-to-2017-stats_processed.csv
[처리중] ./datas/2\1718\england-premier-league-matches-2017-to-2018-stats.csv
[완료] matches: ./datas/2\1718\england-premier-league-matches-2017-to-2018-stats_processed.csv
[처리중] ./datas/2\1819\england-premier-league-matches-2018-to-2019-stats.csv
[완료] matches: ./datas/2\1819\england-premier-league-matches-2018-to-2019-stats_processed.csv
[처리중] ./datas/2\1920\england-premier-league-matches-2019-to-2020-stats.csv
[완료] matches: ./datas/2\1920\england-premier-league-matches-2019-to-2020-stats_processed.csv
[처리중] ./datas/2\2021\england-premier-league-matches-2020-to-2021-stats.csv
[완료] matches: ./datas/2\2021\england-premier-league-matches-2020-to-2021-stats_processed.csv
[처리중] ./datas/2\2122\england-premier-league-matches-2021-to-2022-stats.csv
[완료] matches: ./datas/2\2122\england-premier-league-matches-2021-to-2022-stats_proces

3. 하나로 통합

In [41]:
import pandas as pd
import os

# 통합 저장용 리스트
merged_dfs = []

# 폴더 순회
for root, dirs, files in os.walk(data_root):
    for file in sorted(files):
        if file.endswith('_processed.csv') and 'matches' in file.lower():
            file_path = os.path.join(root, file)
            try:
                df = pd.read_csv(file_path)
                merged_dfs.append(df)
                print(f'[통합 대상] {file_path}')
            except Exception as e:
                print(f'[통합 오류] {file_path}: {e}')

# 하나로 합치고 저장
if merged_dfs:
    df_merged = pd.concat(merged_dfs, ignore_index=True)
    output_path = os.path.join(data_root, 'matches_processed_full.csv')
    df_merged.to_csv(output_path, index=False)
    print(f'[저장 완료] {output_path} ({len(df_merged)} rows)')
else:
    print('[경고] 통합할 파일이 없습니다.')


[통합 대상] ./datas/2\1617\england-premier-league-matches-2016-to-2017-stats_processed.csv
[통합 대상] ./datas/2\1718\england-premier-league-matches-2017-to-2018-stats_processed.csv
[통합 대상] ./datas/2\1819\england-premier-league-matches-2018-to-2019-stats_processed.csv
[통합 대상] ./datas/2\1920\england-premier-league-matches-2019-to-2020-stats_processed.csv
[통합 대상] ./datas/2\2021\england-premier-league-matches-2020-to-2021-stats_processed.csv
[통합 대상] ./datas/2\2122\england-premier-league-matches-2021-to-2022-stats_processed.csv
[통합 대상] ./datas/2\2223\england-premier-league-matches-2022-to-2023-stats_processed.csv
[통합 대상] ./datas/2\2324\england-premier-league-matches-2023-to-2024-stats_processed.csv
[통합 대상] ./datas/2\2425\england-premier-league-matches-2024-to-2025-stats_processed.csv
[저장 완료] ./datas/2\matches_processed_full.csv (3410 rows)


4.

In [42]:
import pandas as pd
import os

N = 3  # 최근 경기 수
full_path = os.path.join(data_root, 'matches_processed_full.csv')
df_full = pd.read_csv(full_path)
df_full['date_GMT'] = pd.to_datetime(df_full['date_GMT'], errors='coerce', format='%b %d %Y - %I:%M%p')

def calculate_head_to_head_winrate(df_full, home, away, date, file_path, n=N):
    h2h_matches = df_full[
        (((df_full['home_team_name'] == home) & (df_full['away_team_name'] == away)) |
         ((df_full['home_team_name'] == away) & (df_full['away_team_name'] == home)))
        & (df_full['date_GMT'] < date)
    ].sort_values('date_GMT', ascending=False).head(n)

    count = len(h2h_matches)
    if count == 0:
        print(f"⚠️ [부족] {file_path} | {home} vs {away} @ {date.date()} → 최근 맞대결 0건")
        return 0.5, 0 
    
    home_win_count = 0
    for _, row in h2h_matches.iterrows():
        if row['home_team_name'] == home and row['home_result'] == 'win':
            home_win_count += 1
        elif row['away_team_name'] == home and row['away_result'] == 'win':
            home_win_count += 1

    return home_win_count / count, count

# 시즌 파일 순회
for root, dirs, files in os.walk(data_root):
    if '1617' in root or 'final' in root.lower():
        continue

    for file in files:
        if file.endswith('_processed.csv') and 'england-premier-league-matches' in file:
            file_path = os.path.join(root, file)
            try:
                print(f"[시즌 처리 중] {file_path}")
                df = pd.read_csv(file_path)
                df['date_GMT'] = pd.to_datetime(df['date_GMT'], errors='coerce', format='%b %d %Y - %I:%M%p')

                home_rates, away_rates, counts = [], [], []

                for _, row in df.iterrows():
                    home_team, away_team, date = row['home_team_name'], row['away_team_name'], row['date_GMT']
                    home_winrate, count = calculate_head_to_head_winrate(df_full, home_team, away_team, date, file_path)
                    home_rates.append(home_winrate)
                    away_rates.append(1 - home_winrate)
                    counts.append(count)

                df[f'home_winrate_recent'] = home_rates
                df[f'away_winrate_recent'] = away_rates
                df[f'recent_count'] = counts

                save_path = os.path.join(root, file.replace('_processed.csv', '_final.csv'))
                df.to_csv(save_path, index=False)
                print(f"[저장 완료] {save_path}")

            except Exception as e:
                print(f"[오류 발생] {file_path}: {e}")


[시즌 처리 중] ./datas/2\1718\england-premier-league-matches-2017-to-2018-stats_processed.csv
⚠️ [부족] ./datas/2\1718\england-premier-league-matches-2017-to-2018-stats_processed.csv | Crystal Palace vs Huddersfield Town @ 2017-08-12 → 최근 맞대결 0건
⚠️ [부족] ./datas/2\1718\england-premier-league-matches-2017-to-2018-stats_processed.csv | Brighton & Hove Albion vs Manchester City @ 2017-08-12 → 최근 맞대결 0건
⚠️ [부족] ./datas/2\1718\england-premier-league-matches-2017-to-2018-stats_processed.csv | Newcastle United vs Tottenham Hotspur @ 2017-08-13 → 최근 맞대결 0건
⚠️ [부족] ./datas/2\1718\england-premier-league-matches-2017-to-2018-stats_processed.csv | Leicester City vs Brighton & Hove Albion @ 2017-08-19 → 최근 맞대결 0건
⚠️ [부족] ./datas/2\1718\england-premier-league-matches-2017-to-2018-stats_processed.csv | Huddersfield Town vs Newcastle United @ 2017-08-20 → 최근 맞대결 0건
⚠️ [부족] ./datas/2\1718\england-premier-league-matches-2017-to-2018-stats_processed.csv | Huddersfield Town vs Southampton @ 2017-08-26 → 최근 맞대결 0건

In [ ]:
5.

In [43]:
def get_position_group(pos_str):
    if pd.isna(pos_str):
        return None
    
    pos_str = pos_str.strip().upper()
    positions = [p.strip() for p in pos_str.split(',')]
    
    counts = {'att':0, 'mid':0, 'def':0, 'gk':0}
    for pos in positions:
        if pos == 'GK':
            counts['gk']+=1
        elif pos == 'AM':
            counts['att'] += 1
        elif pos.endswith('M'):
            counts['mid'] += 1 
        elif pos.endswith('B'):
            counts['def'] += 1
        else:
            counts['att'] += 1
            
    max_count = max(counts.values())
    majority = [k for k, v in counts.items() if v==max_count]
    if len(majority) == 1:
        return majority[0]
    else:
        for pos in positions:
            if pos == 'GK':
                return 'gk'
            elif pos.endswith('M'):
                return 'mid'
            elif pos.endswith('B'):
                return 'def'
            else:
                return 'att'
            
def clean_age(age_str):
    if pd.isna(age_str): return None
    return int(age_str.split('-')[0]) if '-' in age_str else int(age_str)

def process_player_file(path, team_prefix):
    df = pd.read_csv(path)
    df['position_group'] = df['position'].apply(get_position_group)
    df['age'] = df['age'].apply(clean_age)
    
    df = df.drop(columns=[col for col in ['player', 'shirtnumber', 'nationality'] if col in df.columns])
    df = df.dropna(subset=['position_group'])
    
    grouped = df.groupby('position_group').mean(numeric_only=True)
    
    reanamed_list = []    
    for group in grouped.index:
        row = grouped.loc[group]
        row_prefixed = row.add_prefix(f"{team_prefix}_{group}_")
        reanamed_list.append(row_prefixed)
    combined = pd.concat(reanamed_list).to_frame().T
    return combined
    
def process_gk_file(path, team_prefix):
    df = pd.read_csv(path)
    df = df.drop(columns=[col for col in ['player', 'nationality'] if col in df.columns])
    df['age'] = df['age'].apply(clean_age)
    df = df.mean(numeric_only=True).to_frame().T
    df = df.add_prefix(f"{team_prefix}_")
    return df

6. vector 파일 생성

In [44]:

for root, dirs, files in os.walk(data_root2):
    if not any(f.endswith('players_summary.csv') for f in files):
        continue
    
    try:
        home_players_path = next(f for f in files if "home_palyers_summary" in f)
        away_players_path = next(f for f in files if "away_players_summary" in f)
        home_gk_path = next(f for f in files if "home_gk" in f)
        away_gk_path = next(f for f in files if "away_gk" in f)
        
        home_players = process_player_file(os.path.join(root, home_players_path), "home")
        away_players = process_player_file(os.path.join(root, away_players_path), "away")
        home_gk = process_gk_file(os.path.join(root, home_gk_path), "home")
        away_gk = process_gk_file(os.path.join(root, away_gk_path), "away")
        
        ##home_gk = home_gk.drop(columns=[col for col in home_gk.columns if 'gk_goals_against' in col], errors='ignore')
        ##away_gk = away_gk.drop(columns=[col for col in away_gk.columns if 'gk_goals_against' in col], errors='ignore')
        
        final_vector = pd.concat([home_players, away_players, home_gk, away_gk], axis=1)
        date_tag = os.path.basename(root)
        save_path = os.path.join(root, f"{date_tag}_vector.csv")
        final_vector.to_csv(save_path, index=False, encoding="utf-8-sig")
        print(f"[저장 완료] {save_path}")
        
    except Exception as e:
        print(f"[오류 발생]{root} - {e}")

[저장 완료] ./datas/1\2017-2018\Arsenal\20170811\20170811_vector.csv
[저장 완료] ./datas/1\2017-2018\Arsenal\20170909\20170909_vector.csv
[저장 완료] ./datas/1\2017-2018\Arsenal\20170925\20170925_vector.csv
[저장 완료] ./datas/1\2017-2018\Arsenal\20171001\20171001_vector.csv
[저장 완료] ./datas/1\2017-2018\Arsenal\20171028\20171028_vector.csv
[저장 완료] ./datas/1\2017-2018\Arsenal\20171118\20171118_vector.csv
[저장 완료] ./datas/1\2017-2018\Arsenal\20171129\20171129_vector.csv
[저장 완료] ./datas/1\2017-2018\Arsenal\20171202\20171202_vector.csv
[저장 완료] ./datas/1\2017-2018\Arsenal\20171216\20171216_vector.csv
[저장 완료] ./datas/1\2017-2018\Arsenal\20171222\20171222_vector.csv
[저장 완료] ./datas/1\2017-2018\Arsenal\20180103\20180103_vector.csv
[저장 완료] ./datas/1\2017-2018\Arsenal\20180120\20180120_vector.csv
[저장 완료] ./datas/1\2017-2018\Arsenal\20180203\20180203_vector.csv
[저장 완료] ./datas/1\2017-2018\Arsenal\20180301\20180301_vector.csv
[저장 완료] ./datas/1\2017-2018\Arsenal\20180311\20180311_vector.csv
[저장 완료] ./datas/1\2017-20

7. matches_processed + vector 병합

In [45]:
import re
team_folder_map = {
    "AFC Bournemouth" : "Bournemouth",
    "Arsenal" : "Arsenal",
    'Aston Villa' : 'Aston Villa',
    'Brentford' : 'Brentford',
    'Brighton & Hove Albion' : 'Brighton',
    'Burnley' : 'Burnley',
    'Cardiff City' : 'Cardiff City',
    'Chelsea' :'Chelsea',
    'Crystal Palace' : 'Crystal Palace',
    'Everton' : 'Everton',
    'Fulham': 'Fulham',
    'Huddersfield Town' : 'Huddersfield',
    'Ipswich Town' : 'Ipswich Town',
    'Leeds United' : 'Leeds United',
    'Leicester City' : 'Leicester City',
    'Liverpool' : 'Liverpool',
    'Luton Town' : 'Luton Town',
    'Manchester City' : 'Manchester City',
    'Manchester United' : 'Manchester Utd',
    'Newcastle United' : 'Newcastle Utd',
    'Norwich City' : 'Norwich City',
    'Nottingham Forest' : 'Nott\'ham Forest',
    'Sheffield United' : 'Sheffield Utd',
    'Southampton' : 'Southampton',
    'Stoke City' : 'Stoke City',
    'Swansea City' : 'Swansea City',
    'Tottenham Hotspur' : 'Tottenham',
    'Watford' : 'Watford',
    'West Bromwich Albion' : 'West Brom',
    'West Ham United' : 'West Ham',
    'Wolverhampton Wanderers' : 'Wolves'
}

def normalize_date(date_str):
    return pd.to_datetime(date_str).strftime("%Y%m%d")

def extract_season_from_filename(filename):
    match = re.search(r'(\d{4})-to-(\d{4})', filename)
    if match:
        return f"{match.group(1)}-{match.group(2)}"
    return None

def find_team_folder(base_dir, team_name):
    mapped = team_folder_map.get(team_name)
    if mapped:
        candidate = os.path.join(base_dir, mapped)
        if os.path.exists(candidate):
            return mapped
    return None

season_data = {}

for root, dirs, files in os.walk(data_root):
    for file in files:
        if file.endswith('.csv') and 'final' in file.lower():
            file_path = os.path.join(root, file)
            try:
                df = pd.read_csv(file_path)
                season = extract_season_from_filename(file)
                if season is None:
                    print(f"[시즌 추출 실패] {file}")
                    continue
                
                for idx, row in df.iterrows():
                    try:
                        home_team = row['home_team_name']
                        date_tag = normalize_date(row['date_GMT'])
                        season_dir = os.path.join(data_root2, season)
                        team_folder = find_team_folder(season_dir, home_team)
                        
                        if team_folder:
                            vector_path = os.path.join(season_dir, team_folder, date_tag, f"{date_tag}_vector.csv")
                            if os.path.exists(vector_path):
                                vector_df = pd.read_csv(vector_path).dropna(how='all')
                                if not vector_df.empty:
                                    vector_row = vector_df.iloc[0]
                                    combined_row = pd.concat([row, vector_row])
                                    if season not in season_data:
                                        season_data[season] = []
                                    season_data[season].append(combined_row)
                                        
                            else:
                                print(f"[벡터 없음] {vector_path}")
                        else:
                            print(f"[팀 폴더 없음] {home_team} in {season}")
                    except Exception as e:
                        print(f"[행 오류] {idx} - {e}")
            except Exception as e:
                print(f"[파일 오류] {file_path} - {e}")


#시즌별로 저장
for season, rows in season_data.items():
    if rows:
        season_df = pd.DataFrame(rows)
        folder_path = os.path.join(data_root, 'final')
        save_path = os.path.join(folder_path,  f"final_{season}.csv")
        season_df.to_csv(save_path, index=False, encoding='utf-8-sig')
        print(f"[저장 완료] {save_path}")              
    else:
        print(f"[스킵] {season} 시즌 데이터 없음")

[벡터 없음] ./datas/1\2017-2018\Chelsea\20171023\20171023_vector.csv
[벡터 없음] ./datas/1\2017-2018\Southampton\20180120\20180120_vector.csv
[벡터 없음] ./datas/1\2017-2018\Swansea City\20180120\20180120_vector.csv
[시즌 추출 실패] final_2017-2018.csv
[시즌 추출 실패] final_2018-2019.csv
[시즌 추출 실패] final_2019-2020.csv
[시즌 추출 실패] final_2020-2021.csv
[시즌 추출 실패] final_2021-2022.csv
[시즌 추출 실패] final_2022-2023.csv
[시즌 추출 실패] final_2023-2024.csv
[시즌 추출 실패] final_2024-2025.csv
[저장 완료] ./datas/2\final\final_2017-2018.csv
[저장 완료] ./datas/2\final\final_2018-2019.csv
[저장 완료] ./datas/2\final\final_2019-2020.csv
[저장 완료] ./datas/2\final\final_2020-2021.csv
[저장 완료] ./datas/2\final\final_2021-2022.csv
[저장 완료] ./datas/2\final\final_2022-2023.csv
[저장 완료] ./datas/2\final\final_2023-2024.csv
[저장 완료] ./datas/2\final\final_2024-2025.csv


결측값 확인

In [34]:
folder_path = data_root + '/final'
file_list = os.listdir(folder_path)

missing_details = {}     # 파일별 (date, column) 목록
missing_summary = {}     # 파일별 {column: count} 요약

for file in file_list:
    filepath = os.path.join(folder_path, file)

    if file.endswith('.csv'):
        try:
            df = pd.read_csv(filepath)

            if 'date_GMT' not in df.columns:
                print(f"❌ 'date_GMT' column missing in {file}, skipping.")
                continue

            for col in df.columns:
                null_count = df[col].isnull().sum()
                if null_count > 0:
                    # 세부 목록 저장
                    missing_rows = df[df[col].isnull()]
                    for _, row in missing_rows.iterrows():
                        date = row['date_GMT']
                        missing_details.setdefault(file, []).append((date, col))
                    
                    # 요약 정보 저장
                    missing_summary.setdefault(file, {})[col] = null_count

        except Exception as e:
            print(f"⚠️ Error reading {file}: {e}")

# 출력
for file in missing_details:
    print(f"\n📂 파일: {file}")
    
    # 요약 정보 출력
    print("📊 결측 요약:")
    for col, count in missing_summary[file].items():
        print(f" - {col}: {count}개 결측")

    # 세부 정보 출력
    print("📅 결측 발생 위치:")
    for date, col in missing_details[file]:
        print(f"   → {date}  |  {col}")


📂 파일: final_2017-2018.csv
📊 결측 요약:
 - home_gk_save_pct: 18개 결측
 - home_gk_pct_goal_kicks_launched: 1개 결측
 - home_gk_goal_kick_length_avg: 1개 결측
 - home_gk_avg_distance_def_actions: 20개 결측
 - away_gk_save_pct: 11개 결측
 - away_gk_crosses_stopped_pct: 1개 결측
 - away_gk_avg_distance_def_actions: 13개 결측
📅 결측 발생 위치:
   → 2017-08-12 14:00:00  |  home_gk_save_pct
   → 2017-08-27 12:30:00  |  home_gk_save_pct
   → 2017-08-27 15:00:00  |  home_gk_save_pct
   → 2017-09-09 11:30:00  |  home_gk_save_pct
   → 2017-09-16 16:30:00  |  home_gk_save_pct
   → 2017-09-23 14:00:00  |  home_gk_save_pct
   → 2017-10-21 14:00:00  |  home_gk_save_pct
   → 2017-10-21 14:00:00  |  home_gk_save_pct
   → 2017-10-28 14:00:00  |  home_gk_save_pct
   → 2017-11-18 15:00:00  |  home_gk_save_pct
   → 2017-11-29 19:45:00  |  home_gk_save_pct
   → 2018-01-13 17:30:00  |  home_gk_save_pct
   → 2018-02-03 15:00:00  |  home_gk_save_pct
   → 2018-03-04 16:00:00  |  home_gk_save_pct
   → 2018-03-10 15:00:00  |  home_gk_save_pct

8. 접두어 통일

In [ ]:
import pandas as pd
import os

# 접두어 붙일 컬럼
home_team_features = [
    'Pre-Match PPG (Home)', 'Home Team Pre-Match xG', 'team_a_xg', 'odds_ft_home_team_win'
]

away_team_features = [
    'Pre-Match PPG (Away)', 'Away Team Pre-Match xG', 'team_b_xg', 'odds_ft_away_team_win'
]

# 삭제할 컬럼
delete_columns = [
    'total_goal_count', 'total_goals_at_half_time', 'average_goals_per_match_pre_match', 'btts_percentage_pre_match',
    'over_15_percentage_pre_match', 'over_25_percentage_pre_match', 'over_35_percentage_pre_match', 'over_45_percentage_pre_match',
    'over_15_HT_FHG_percentage_pre_match', 'over_05_HT_FHG_percentage_pre_match', 'over_15_2HG_percentage_pre_match', 'over_05_2HG_percentage_pre_match',
    'average_corners_per_match_pre_match', 'average_cards_per_match_pre_match',
    'odds_ft_home_team_win', 'odds_ft_draw', 'odds_ft_away_team_win',
    'odds_ft_over15', 'odds_ft_over25', 'odds_ft_over35', 'odds_ft_over45',
    'odds_btts_yes', 'odds_btts_no'
]

# 폴더 경로
folder_path = os.path.join(data_root, 'final')
file_list = os.listdir(folder_path)

for file in file_list:
    filepath = os.path.join(folder_path, file)

    # ✅ 실제로 CSV 파일만 처리하도록 수정
    if not file.endswith('.csv'):
        continue

    try:
        df = pd.read_csv(filepath)

        # 삭제
        df = df.drop(columns=[col for col in delete_columns if col in df.columns], errors='ignore')

        # 컬럼 이름 변경
        rename_dict = {}
        for col in df.columns:
            if col in home_team_features:
                rename_dict[col] = f'home_{col}'
            elif col in away_team_features:
                rename_dict[col] = f'away_{col}'

        df.rename(columns=rename_dict, inplace=True)

        # 저장 (덮어쓰기)
        df.to_csv(filepath, index=False, encoding='utf-8-sig')
        print(f'✅ 완료: {file}')

    except Exception as e:
        print(f"⚠️ 오류: {file} → {e}")
        

✅ 완료: final_2017-2018.csv
✅ 완료: final_2018-2019.csv
✅ 완료: final_2019-2020.csv
✅ 완료: final_2020-2021.csv
✅ 완료: final_2021-2022.csv
✅ 완료: final_2022-2023.csv
✅ 완료: final_2023-2024.csv
✅ 완료: final_2024-2025.csv
⚠️ 확인 필요: final_2017-2018.csv
  ❌ 누락된 home_ 컬럼: ['home_odds_ft_home_team_win']
  ❌ 누락된 away_ 컬럼: ['away_odds_ft_away_team_win']
⚠️ 확인 필요: final_2018-2019.csv
  ❌ 누락된 home_ 컬럼: ['home_odds_ft_home_team_win']
  ❌ 누락된 away_ 컬럼: ['away_odds_ft_away_team_win']
⚠️ 확인 필요: final_2019-2020.csv
  ❌ 누락된 home_ 컬럼: ['home_odds_ft_home_team_win']
  ❌ 누락된 away_ 컬럼: ['away_odds_ft_away_team_win']
⚠️ 확인 필요: final_2020-2021.csv
  ❌ 누락된 home_ 컬럼: ['home_odds_ft_home_team_win']
  ❌ 누락된 away_ 컬럼: ['away_odds_ft_away_team_win']
⚠️ 확인 필요: final_2021-2022.csv
  ❌ 누락된 home_ 컬럼: ['home_odds_ft_home_team_win']
  ❌ 누락된 away_ 컬럼: ['away_odds_ft_away_team_win']
⚠️ 확인 필요: final_2022-2023.csv
  ❌ 누락된 home_ 컬럼: ['home_odds_ft_home_team_win']
  ❌ 누락된 away_ 컬럼: ['away_odds_ft_away_team_win']
⚠️ 확인 필요: final_2023-202

Final. 최근 n경기 평균값들 dataset

In [52]:
from tqdm import tqdm

N = 3

folder_path = os.path.join(data_root, 'final')
save_path_root = './datas/input'

for file in os.listdir(folder_path):
    if not file.endswith('.csv'):
        continue
    
    file_path = os.path.join(folder_path, file)
    
    try:
        season = file.replace('final_', '').replace('.csv', '')
        df = pd.read_csv(file_path)
        df['date'] = pd.to_datetime(df['date_GMT'], errors='coerce')
        df = df.sort_values('date')

        # 설정
        target_cols = ['home_team_goal_count', 'away_team_goal_count']
        meta_cols = ['home_result', 'away_result', 'home_winrate_recent', 'away_winrate_recent', 'recent_count']
        exclude_cols = ['date_GMT', 'home_team_name', 'away_team_name', 'date'] + target_cols + meta_cols
        feature_cols = [col for col in df.columns if col not in exclude_cols]

        home_feature_cols = [col for col in feature_cols if col.startswith('home_')]
        away_feature_cols = [col for col in feature_cols if col.startswith('away_')]
         
        # 결과 저장
        final_rows = []

        for idx, row in tqdm(df.iterrows(), total=len(df)):
            current_date = row['date']
            home_team = row['home_team_name']
            away_team = row['away_team_name']

            # 홈팀의 최근 홈경기 N경기
            home_past = df[
                (df['home_team_name'] == home_team) &
                (df['date'] < current_date)
            ].sort_values('date', ascending=False).head(N)

            # 어웨이팀의 최근 어웨이경기 N경기
            away_past = df[
                (df['away_team_name'] == away_team) &
                (df['date'] < current_date)
            ].sort_values('date', ascending=False).head(N)

            if home_past.empty or away_past.empty:
                continue

            # 평균값 계산
            home_mean = home_past[home_feature_cols].mean()
            away_mean = away_past[away_feature_cols].mean()

            # 라벨 및 기타 메타 정보 유지
            label_data = row[target_cols].copy()
            label_data['date'] = row['date']
            label_data['home_team'] = home_team
            label_data['away_team'] = away_team

            meta_data = {
                'home_winrate_recent' : row['home_winrate_recent'],
                'away_winrate_recent' : row['away_winrate_recent'],
                'recent_count' : row['recent_count'],
                'home_result' : row['home_result'],
                'away_result' : row['away_result']
            }
            
            # 최종 벡터 생성
            final_row = pd.concat([label_data, pd.Series(meta_data), home_mean, away_mean])
            final_rows.append(final_row)
            
        if final_rows:     
            # DataFrame 생성 및 저장
            final_df = pd.DataFrame(final_rows)
            save_path = os.path.join(save_path_root, f'input_{season}.csv')
            final_df.to_csv(save_path, index=False, encoding='utf-8-sig')
            print(f"저장 완료: {save_path}")
        else:
            print(f"[스킵 {season} - 3경기 미만 경기만 존재]")
    except Exception as e:
        print(f'[오류] {file_path} - {e}')


100%|██████████| 377/377 [00:00<00:00, 427.66it/s]


저장 완료: ./datas/input\input_2017-2018.csv


100%|██████████| 380/380 [00:00<00:00, 421.48it/s]


저장 완료: ./datas/input\input_2018-2019.csv


100%|██████████| 380/380 [00:00<00:00, 442.88it/s]


...

저장 완료: ./datas/input\input_2022-2023.csv


100%|██████████| 380/380 [00:00<00:00, 446.57it/s]


저장 완료: ./datas/input\input_2023-2024.csv


100%|██████████| 370/370 [00:00<00:00, 442.58it/s]


저장 완료: ./datas/input\input_2024-2025.csv


In [ ]:
#DB 연결
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1234',
    database='premo',
)
cursor = conn.cursor()

#team_name -> team_id 매핑
team_df = pd.read_sql("SELECT team_id, team_name FROM teams", conn)
team_map = dict(zip(team_df['team_name'], team_df['team_id']))

#position_name -> position_id 매핑
position_df = pd.read_sql("SELECT position_id, position_name FROM positions", conn)
position_map = dict(zip(position_df['position_name'], position_df['position_id']))

for root, dirs, files in os.walk(data_root):
    for file in files:
        if file.endswith('.csv') and 'processed' in file.lower():
            file_path = os.path.join(root, file)
            try:
                df = pd.read_csv(file_path)
                
                #matches 파일일 경우
                if 'matches' in file.lower():
                    for col in ['home_team_name', 'away_team_name']:
                        if col in df.columns:
                            df[col] = df[col].map(team_map)
                
                 #players 파일일 경우
                if 'players' in file.lower():
                    if 'position' in df.columns:
                        df['position'] = df['position'].map(position_map)
                
                if 'team_name' in df.columns:
                    df['team_name'] = df['team_name'].map(team_map)
                    
                df.to_csv(file_path, index=False)
                print(f"[완료] id 매핑: {file_path}")
            except Exception as e:
                print(f"[오류] {file_path}: {e}")
           
#완료
conn.commit()
cursor.close()
conn.close()

[완료] id 매핑: ./datas\1011\england-premier-league-matches-2010-to-2011-stats_processed.csv
[완료] id 매핑: ./datas\1011\england-premier-league-merged_teams-2010-to-2011-stats_processed.csv


C:\Users\user080524\AppData\Local\Temp\ipykernel_9008\1670578138.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  team_df = pd.read_sql("SELECT team_id, team_name FROM teams", conn)
C:\Users\user080524\AppData\Local\Temp\ipykernel_9008\1670578138.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  position_df = pd.read_sql("SELECT position_id, position_name FROM positions", conn)


[완료] id 매핑: ./datas\1011\england-premier-league-players-2010-to-2011-stats_processed.csv
[완료] id 매핑: ./datas\1112\england-premier-league-matches-2011-to-2012-stats_processed.csv
[완료] id 매핑: ./datas\1112\england-premier-league-merged_teams-2011-to-2012-stats_processed.csv
[완료] id 매핑: ./datas\1112\england-premier-league-players-2011-to-2012-stats_processed.csv
[완료] id 매핑: ./datas\1213\england-premier-league-matches-2012-to-2013-stats_processed.csv
[완료] id 매핑: ./datas\1213\england-premier-league-merged_teams-2012-to-2013-stats_processed.csv
[완료] id 매핑: ./datas\1213\england-premier-league-players-2012-to-2013-stats_processed.csv
[완료] id 매핑: ./datas\1213\~$england-premier-league-matches-2012-to-2013-stats_processed.csv
[완료] id 매핑: ./datas\1314\england-premier-league-matches-2013-to-2014-stats_processed.csv
[완료] id 매핑: ./datas\1314\england-premier-league-merged_teams-2013-to-2014-stats_processed.csv
[완료] id 매핑: ./datas\1314\england-premier-league-players-2013-to-2014-stats_processed.csv
[완료]